In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e2/sample_submission.csv
/kaggle/input/playground-series-s5e2/train.csv
/kaggle/input/playground-series-s5e2/test.csv
/kaggle/input/playground-series-s5e2/training_extra.csv


In [42]:
train = pd.read_csv('/kaggle/input/playground-series-s5e2/train.csv', index_col=['id'])
test = pd.read_csv('/kaggle/input/playground-series-s5e2/test.csv', index_col=['id'])
train_xtra = pd.read_csv('/kaggle/input/playground-series-s5e2/test.csv', index_col=['id'])

test

,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg)
id,,,,,,,,,
300000,Puma,Leather,Small,2.0,No,No,Tote,Green,20.671147
300001,Nike,Canvas,Medium,7.0,No,Yes,Backpack,Green,13.564105
300002,Adidas,Canvas,Large,9.0,No,Yes,Messenger,Blue,11.809799
300003,Adidas,Nylon,Large,1.0,Yes,No,Messenger,Green,18.477036
300004,NaN,Nylon,Large,2.0,Yes,Yes,Tote,Black,9.907953
...,...,...,...,...,...,...,...,...,...
499995,Adidas,Canvas,Large,2.0,Yes,No,Messenger,Red,7.383498
499996,Nike,Polyester,Small,9.0,No,Yes,Messenger,Pink,6.058394
499997,Jansport,Nylon,Small,9.0,No,Yes,Tote,Green,26.890163


In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 300000 entries, 0 to 299999
Data columns (total 10 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   Brand                 290295 non-null  object 
 1   Material              291653 non-null  object 
 2   Size                  293405 non-null  object 
 3   Compartments          300000 non-null  float64
 4   Laptop Compartment    292556 non-null  object 
 5   Waterproof            292950 non-null  object 
 6   Style                 292030 non-null  object 
 7   Color                 290050 non-null  object 
 8   Weight Capacity (kg)  299862 non-null  float64
 9   Price                 300000 non-null  float64
dtypes: float64(3), object(7)
memory usage: 25.2+ MB


*Create a pairplot*

*Get numeric columns and categorical columns*

In [4]:
num_df = train.select_dtypes(include=['number']).drop('Price', axis=1)
cat_df = train.select_dtypes(include=['object'])

num_cols = num_df.columns
cat_cols = cat_df.columns

num_df

,Compartments,Weight Capacity (kg)
id,,
0,7.0,11.611723
1,10.0,27.078537
2,2.0,16.643760
3,8.0,12.937220
4,1.0,17.749338
...,...,...
299995,9.0,12.730812
299996,6.0,26.633182
299997,9.0,11.898250


*Seperate categorical columns into ordinal (to be ordinal encoded) and non-ordinal (to be one-hot encoded)*

# TODO: Try changing the ordinal and non-ordinal columns around

In [5]:
ord_cols = ['Size', 'Laptop Compartment', 'Waterproof']
oh_cols = ['Brand', 'Material', 'Style', 'Color']

ord_df = train[ord_cols]
oh_df = train[oh_cols]

*Get unique values*

In [6]:
for col in ord_df.columns:
    print(f'col: {col}\n\
values: {ord_df[col].unique()}')

col: Size
values: ['Medium' 'Small' 'Large' nan]
col: Laptop Compartment
values: ['Yes' 'No' nan]
col: Waterproof
values: ['No' 'Yes' nan]


*Get the ordering of columns in the `train` DataFrame*

In [7]:
for i, col in enumerate(train.columns):
    print(f'{i}, {col}')

0, Brand
1, Material
2, Size
3, Compartments
4, Laptop Compartment
5, Waterproof
6, Style
7, Color
8, Weight Capacity (kg)
9, Price


 *Define ordering for each value*

In [8]:
size_categories = ['Small', 'Medium', 'Large']
yesno_categories = ['No', 'Yes']

*Create transformers for each type of column*

In [10]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler, FunctionTransformer

# Numerical columns

num_transformer = Pipeline(
    steps=[
        ('num_impute', SimpleImputer(strategy='median')),
        ('scale', StandardScaler()),
    ]
)

num_df = pd.DataFrame(num_transformer.fit_transform(num_df))
num_df.columns = num_cols

# For ordinal columns

ord_preprocessor = ColumnTransformer(
    transformers=[
        ('size_encoder', OrdinalEncoder(categories=[size_categories], ), ['Size']),
        ('laptop_encoder', OrdinalEncoder(categories=[yesno_categories]), ['Laptop Compartment']),
        ('wp_encoder', OrdinalEncoder(categories=[yesno_categories]), ['Waterproof'])
    ],
    remainder='passthrough'
)

ord_transformer = Pipeline(
    steps=[
        ('ord_impute', SimpleImputer(strategy='most_frequent')),
        ('df_conversion', FunctionTransformer(lambda x: pd.DataFrame(x, columns=ord_cols))),
        ('preprocess', ord_preprocessor)
    ]
)

ord_df = pd.DataFrame(ord_transformer.fit_transform(ord_df), columns = ord_cols)

# One hot columns

oh_transformer = Pipeline(
    steps=[
        ('oh_impute', SimpleImputer(strategy='most_frequent')),
        ('oh_encode', OneHotEncoder(handle_unknown='ignore', sparse=False))
    ]
)

oh_df = pd.DataFrame(oh_transformer.fit_transform(oh_df))



/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [44]:
from sklearn.model_selection import train_test_split
import xgboost as xgb

my_model = xgb.XGBRegressor(objective='reg:squarederror',
                        n_estimators=500)

preprocessor = ColumnTransformer(
    transformers = [
        ('num', num_transformer, num_cols),
        ('ord', ord_transformer, ord_cols),
        ('oh', oh_transformer, oh_cols)
    ],
    remainder = 'passthrough'
)

pipeline = Pipeline(
    steps = [
        ('preprocessor', preprocessor),
        ('predict', my_model)
    ]
)

X = train.drop(columns = ['Price'])
y = train['Price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.fit_transform(X_test)
my_model.fit(X_train, y_train,
            early_stopping_rounds = 5,
            eval_set=[(X_test, y_test)])

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


[0]	validation_0-rmse:38.93360
[1]	validation_0-rmse:38.93118
[2]	validation_0-rmse:38.93056
[3]	validation_0-rmse:38.92842
[4]	validation_0-rmse:38.92782
[5]	validation_0-rmse:38.93134


/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[6]	validation_0-rmse:38.93191
[7]	validation_0-rmse:38.93468
[8]	validation_0-rmse:38.93650
[9]	validation_0-rmse:38.93825


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=500, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [ ]:
my_model.p